# 建議先從rag-faiss.ipynb開始跑 環境最複雜 但是弄完 4份都可以跑

必要

miniconda_python(3.10版本):

https://docs.conda.io/projects/miniconda/en/latest/miniconda-other-installer-links.html

裝完打開anaconda prompt:

做一個環境```conda create -n VScode-env-3.10.13 python=3.10.13```

做完啟動```conda activate VScode-env-3.10.13```

安裝faiss插件

```conda install -c pytorch faiss-cpu=1.7.4 mkl=2021 blas=1.0=mkl```

```conda install -c pytorch/label/nightly faiss-cpu```

關掉prompt 重新開啟VScode 右上角kernel 選擇another kernel->python env->VScode-env-3.10.13

插件:

沒裝pandas:```pip install pandas```

沒裝openai_SDK使用:```pip install openai```

沒設定key使用:```setx OPENAI_API_KEY “<yourkey>”```

```echo %OPENAI_API_KEY%```可以看你的KEY

讀embeddings資料

並把dataframe轉為numpy

In [35]:
import pandas as pd
import numpy as np

csv_file = 'data\Qembeddings.csv'
df = pd.read_csv(csv_file)
data = df.to_numpy()
df.head(5)

,0,1,2,3,4,5,6,7,8,9,...,1526,1527,1528,1529,1530,1531,1532,1533,1534,1535
0,-0.022805,-0.003135,-0.013681,-0.036727,-0.044497,0.029007,-0.024501,-0.041485,-0.005907,-0.009333,...,-0.001190,-0.021287,0.009650,-0.010143,-0.028197,0.025374,-0.010055,-0.003249,-0.007062,-0.028197
1,-0.016840,-0.011369,-0.003460,-0.017197,-0.024066,0.012614,-0.030361,-0.040702,0.012908,-0.003005,...,0.013699,-0.041800,0.021296,-0.006463,-0.004542,-0.020491,-0.033731,0.010705,0.003454,-0.024437
2,-0.020366,-0.011115,0.016573,-0.021214,-0.017512,-0.002729,-0.011475,-0.030523,0.020134,-0.016496,...,0.023915,-0.022757,0.027797,-0.001901,-0.010009,0.003963,-0.034329,-0.025637,0.005378,-0.002792
3,-0.029621,-0.031300,-0.004523,-0.022191,-0.040698,0.019835,-0.013771,-0.023770,0.014309,-0.008721,...,0.015738,-0.005817,0.005927,0.000405,-0.026338,0.000488,-0.020086,-0.000731,0.001592,-0.020474
4,-0.014236,-0.011760,-0.014080,-0.002142,-0.036923,0.014444,-0.035155,-0.033881,0.009361,-0.001141,...,0.014509,-0.018397,0.017265,-0.009296,-0.004875,-0.010771,-0.011649,0.004901,0.003660,-0.035259


確認內容

In [36]:
print(data.shape[0])
print(type(data))
print(data.dtype)
print(data[0][0])

669
<class 'numpy.ndarray'>
float64
-0.0228053741157054


寫入維度 並把data加入faiss的index

In [37]:
import faiss

index = faiss.IndexFlatL2(data.shape[1])
index.add(data)

# 問題輸入在此

In [38]:
input = "hey flora! Im having some issue using Instagram! can you teach me how to add friend?"

做問題的embedding

In [39]:
import os
from openai import OpenAI
client = OpenAI()
OpenAI.api_key = os.getenv('OPENAI_API_KEY')

response = client.embeddings.create(
    input=input,
    model="text-embedding-ada-002"
)
question = np.array(response.data[0].embedding, ndmin=2)
print(question)

[[-0.0192867   0.00081752 -0.0193528  ... -0.00241414  0.00399548
  -0.05195116]]


TOP-k的設定並查找問題

In [40]:
k = 3

distances, indices = index.search(question, k)

拿答案

In [41]:
csv_question = 'data\manual.csv'
qdf = pd.read_csv(csv_question)
qarray = qdf.to_numpy()

看答案和距離

In [42]:
answer = [(qarray[i], float(dist)) for dist, i in zip(distances[0], indices[0])]
print(answer)

[(array(["SoI'm trying to add Facebook friends on flora but I just don't get the option , can you please assist ?",
       "Currently, we are dropping the Facebook option due to some security reasons. If you want to add your friend, an alternative way is to add his/her account to your contact info and use the 'Import from Contact' function."],
      dtype=object), 0.26772311329841614), (array(['Are there any plans to make a Flora instagram profile? because I think it would help boost your popularity',
       'Currently we do not have an official Instagram page. We are considering opening an Instagram page and might plan to open one in one day. '],
      dtype=object), 0.31527459621429443), (array(["I'm a flora user, and I really like the new version of flora! However, is there a way to remove certain friends? I think some of my friends were imported from Facebook since my flora account is linked to Facebook. But I would like to keep only some close friends in flora.",
       'To remove

In [43]:
for i in range(0,k):
    print(answer[i][0][1])


Currently, we are dropping the Facebook option due to some security reasons. If you want to add your friend, an alternative way is to add his/her account to your contact info and use the 'Import from Contact' function.
Currently we do not have an official Instagram page. We are considering opening an Instagram page and might plan to open one in one day. 
To remove a friend, you could swipe him/her to the left on the Friend Page, and click it again for confirmation.


只顯示min_distance以內的結果
要試多點無關問題
我猜大概0.25 學弟要幫注意

In [44]:
min_distance = 0.25
for i in range(0,k):
    if answer[i][1]<min_distance:
        print(answer[i][0][1])
